# Radar observations : rainfall composite product, rainfall

The aim of this notebook is to help you read the rainfall data.

For each radar data type, you will find one archive per month, each one sliced in periods of 10 or 11 days (each month is separated in 3 files). 

Each `.npz` file contains 3 arrays :

* `data` : an array containing maps of radar data values.

* `dates` : an array containing a datetime object and indicating the date of each `data` map.

* `miss_dates` : an array containing a datetime object for each missing radar map over the period.

Here, the `data` array contains a rainfall map for each time step. For each pixel, the value represents the cumulated rainfall in hundreth of millimeters (10<sup>-2</sup> mm). If the value is missing then the value is -1.

The time step between two radar scans is 5 min. The radar starts scanning at 00h every day and ends at 23h55. So each file of 11 days should contain a maximum of 3168 rainfall maps (minus the missing dates).

The radar's spatial resolution is 0.01° and the projection system used is EPSG:4326.

You will also find a `radar_coords_ZONE.npz` file containing 2 arrays `lats` and `lons`, indicating the coordinates (latitude and longitude) of the center of each radar pixel.

For more information about data, cf [documentation](https://meteofrance.github.io/meteonet/english/data/rain-radar/).

# Note

<font size="4.5">To use <span style="color:blue">**Cartopy**</span>, a library to plot data with basemaps (see cells below), it is necessary to <span style="color:red">activate the internet connection</span> of that notebook (in edit mode, you can find on the right column, in the *Settings* section, a row entitled *Internet*, put the slider bar on **on**).  </font>

# Quickstart

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

# Input data files are available in the "../input/" directory.
# Any results you write to the current directory are saved as output.

First, let's select the zone and date we want to study : 

In [ ]:
zone = "NW"
year = 2016
month = 5
part_month = 3 # Choice between 1,2,3, as we said earlier each month is divided in 3 parts

Now, we load the data and the coordinates :

In [ ]:
directory = '/kaggle/input/meteonet/'
fname = directory + f'{zone}_rainfall_{str(year)}/{zone}_rainfall_{str(year)}/rainfall-{zone}-{str(year)}-{str(month).zfill(2)}/rainfall-{zone}-{str(year)}-{str(month).zfill(2)}/rainfall_{zone}_{str(year)}_{str(month).zfill(2)}.{str(part_month)}.npz'
fname_coords = directory + 'Radar_coords/Radar_coords/'+ f'radar_coords_{zone}.npz'

d = np.load(fname, allow_pickle=True)
data = d['data']
dates = d['dates']
miss_dates = d['miss_dates']

coords = np.load(fname_coords, allow_pickle=True)
#it is about coordinates of the center of pixels 
lat = coords['lats']
lon = coords['lons']

Let's plot some examples :

In [ ]:
title = "4 examples of rainfall plots"
fig, ax = plt.subplots(2, 2,figsize=(9,9))
fig.suptitle(title, fontsize=16)

# Choose the colormap
cmap = colors.ListedColormap(['silver','white', 'darkslateblue', 'mediumblue','dodgerblue', 
                              'skyblue','olive','mediumseagreen','cyan','lime','yellow',
                              'khaki','burlywood','orange','brown','pink','red','plum'])
bounds = [-1,0,2,4,6,8,10,15,20,25,30,35,40,45,50,55,60,65,75]
norm = colors.BoundaryNorm(bounds, cmap.N)

pl=ax[0,0].pcolormesh(lon, lat, data[0,:,:],cmap=cmap, norm=norm)
ax[0,0].set_ylabel('latitude (degrees_north)')
ax[0,0].set_title(str(dates[0]) + " - "+  zone + " zone")

pl=ax[0,1].pcolormesh(lon, lat, data[1,:,:],cmap=cmap, norm=norm)
ax[0,1].set_title(str(dates[1]) + " - "+  zone + " zone")

pl=ax[1,0].pcolormesh(lon, lat, data[2,:,:],cmap=cmap, norm=norm)
ax[1,0].set_xlabel('longitude (degrees_east)')
ax[1,0].set_ylabel('latitude (degrees_north)')
ax[1,0].set_title(str(dates[2]) + " - "+  zone + " zone")

pl=ax[1,1].pcolormesh(lon, lat, data[3,:,:],cmap=cmap, norm=norm)
ax[1,1].set_xlabel('longitude (degrees_east)')
ax[1,1].set_title(str(dates[3]) + " - "+  zone + " zone")

# Plot the color bar
cbar = fig.colorbar(pl,ax=ax.ravel().tolist(),cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, 
                orientation= 'vertical').set_label('Rainfall (in 1/100 mm) / -1 : missing values')
plt.show()

## Exploring the data

### Data

Dimensions of the ```data``` array: 

* Number of records : the radar records a scan every 5 min

* Number of points on the y axis : latitude

* Number of points on the x axis : longitude

In [ ]:
data.shape

### Dates

Dimension of the ```dates``` array: 

* Number of records : the radar records a scan every 5 min

In [ ]:
dates.shape

In [ ]:
dates[:5]

### Missing dates

Dimension of the ```miss_dates``` array: 

* Number of missing records

In [ ]:
miss_dates.shape

### Latitude

In [ ]:
lat.shape

In [ ]:
lat

# Plot with Cartopy

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# Coordinates of studied area boundaries (in °N and °E)
lllat = 46.25  #lower left latitude
urlat = 51.896  #upper right latitude
lllon = -5.842  #lower left longitude
urlon = 2  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

fig = plt.figure(figsize=(9,10))

# Select projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data
plt.imshow(data[2592+24], interpolation='none', origin='upper',cmap=cmap, norm=norm, extent=extent)

# Add coastlines and borders
ax.coastlines(resolution='50m', linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Add the colorbar
plt.colorbar(cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, 
             orientation= 'horizontal').set_label('Rainfall (in 1/100 mm) / -1 : missing values')
plt.title("Rainfalls - "+ str(dates[2592+24]) + " - "+  zone + " zone")
plt.show()